## Import Statements

## Load data (WMAP, PICO, SNe Ia)

## Posterior Probability Function for WMAP

In [ ]:
#Determine physical bounds on the prior

## WMAP MCMC Fitting

In [4]:
#Check tau_f to tune MCMC, show corner plots of results
#Plot the posterior fit with the data
#Convert omega*h^2 values to omega

## Posterior Probability Function for SNe Ia

## SNe Ia MCMC Fitting

In [5]:
#Check tau_f to tune MCMC, show corner plots of results
#Plot the posterior fit with the data

#Calculate posterior for omega_k = 1 - omega_m - omega_de

## Overlay comparison of WMAP and SNe Ia Posterior distributions

In [7]:
#Overlay the posteriors for the omegas to compare between the two measurements
#Maybe include best fit values from Planck?